In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import ast
import numpy as np
import platform

In [2]:
def make_price_dataframe(code, timeframe, starttime, endtime):
  url = f"https://api.finance.naver.com/siseJson.naver?symbol={code}&requestType=1&startTime={starttime}&endTime={endtime}&timeframe={timeframe}"
  r = requests.get(url)
  soup = BeautifulSoup(r.text, "html.parser")
  price_data_str = str(soup).replace("\t","").replace("\n","").replace(" ","")
  price_data = ast.literal_eval(price_data_str)
  price_data = price_data[1:]
  date_list = []
  market_list = []
  closing_list = []
  high_list = []
  low_list = []
  volume_list = []
  foreign_list = []
  for item in price_data:
    date_list.append(item[0])
    market_list.append(item[1])
    high_list.append(item[2])
    low_list.append(item[3])
    closing_list.append(item[4])
    volume_list.append(item[5])
    foreign_list.append(item[6])
  price_df = pd.DataFrame({('market_price',code):market_list, ('high_price',code):high_list, 
                          ('low_price',code):low_list, ('closing_price',code):closing_list,  
                          ('trading_volume',code):volume_list, ('foreign',code):foreign_list,},
                          index=date_list)
  return price_df


def make_code(x):
    x = str(x)
    return "0"*(6-len(x)) + x


In [3]:
path = pd.read_csv("korea_etf.csv")
code_data = path[["단축코드","한글종목약명"]].copy()
code_data["단축코드"] = code_data["단축코드"].apply(make_code)

------------
# 날짜 수정(필수!!)

In [ ]:
start = '20211102'
end = '20211121'

--------

# 데이터 크롤링(시간 오래걸림)

In [4]:
for num, code in enumerate(code_data["단축코드"]):
  try:
    print(num, code)
    time.sleep(1)
    try:
      price_df = make_price_dataframe(code,"day",start, end)
    except requests.exceptions.Timeout:
      time.sleep(60)
      price_df = make_price_dataframe(code,"day",start, end)
    if num == 0:
      total_price = price_df
    else:
      total_price = pd.merge(total_price,price_df,how='outer',right_index=True,left_index=True)
  except ValueError:
    print("valueError")
    continue
  except KeyError:
    print("keyError")
    continue

0 292340
1 159800
2 361580
3 285000
4 287300
5 287310
6 290080
7 284980
8 287320
9 287330
10 252400
11 252420
12 252410
13 284990
14 285010
15 148020
16 285020
17 315480
18 105780
19 290130
20 368200
21 367760
22 367770
23 388280
24 326240
25 385560
26 385550
27 385540
28 270800
29 307010
30 319870
31 292050
32 342600
33 342500
34 342620
35 342610
36 403990
37 234310
38 241390
39 401170
40 300640
41 266160
42 282000
43 114100
44 295020
45 295000
46 397410
47 397420
48 276650
49 375270
50 399580
51 336160
52 326230
53 272560
54 196230
55 315960
56 252730
57 252720
58 379780
59 219390
60 354240
61 368590
62 267490
63 267500
64 267450
65 267440
66 388420
67 140570
68 140580
69 379790
70 183710
71 310080
72 174360
73 136340
74 281990
75 272570
76 250730
77 291680
78 371150
79 183700
80 278240
81 275750
82 270810
83 361590
84 302450
85 334700
86 334690
87 253280
88 253290
89 225130
90 332930
91 304780
92 306520
93 293180
94 395290
95 395280
96 368190
97 402460
98 395270
99 367740
100 381560

In [5]:
index = []
new_code = []
for col in total_price.columns:
    index.append(col[0])
    new_code.append('A' + col[1])

total_price.columns = [index,new_code]

# 파일 처음 만들 때

In [ ]:
# 주석 지워 사용
# total_price.to_csv('etf_price.csv')

# 원래 파일에 붙일 때

In [ ]:
# 주석 지워 사용
# old_price = pd.read_csv('etf_price.csv', header=[0,1], index_col=0)
# pd.concat([old_price, total_price]).to_csv('etf_price.csv')